# Skip Connections

![skip connection](http://chaosmail.github.io/images/deep-learning/residual.png)

These can be used as part of your repeated block.
The stride cannot be part of the inner operations, otherwise the two tensors would not be the same shape.

The image above shows addition, however concatenation is more common.

In [ ]:
from typing import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim

import numpy as np
from torchvision import datasets, transforms

from tqdm import tqdm

import PIL

In [ ]:
class SkipSequential(nn.Sequential):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_prime = super().forward(x)
        return torch.cat([x, x_prime], dim=1)

In [ ]:
def repeating_block(in_channels: int, out_channels: int) -> nn.Module:
    return nn.Sequential(
        SkipSequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=(3, 3),
                padding=(1, 1),
                padding_mode='reflect',
            ),
            nn.Tanh(),
            nn.Conv2d(
                in_channels=out_channels,
                out_channels=out_channels,
                kernel_size=(3, 3),
                padding=(1, 1),
                padding_mode='reflect',
            ),
        ),
        nn.Tanh(),
        nn.Conv2d(
            in_channels=in_channels + out_channels, # THIS HAS CHANGED
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(2, 2),
            padding=(1, 1),
            padding_mode='reflect',
        ),
        nn.Tanh(),
    )

The important thing to realise here is that the number of channels grows immediately after the skip connection.

If you do forget this then you'll see an error like:

```
RuntimeError: Given groups=1, weight of size [16, 16, 3, 3], expected input[128, 24, 19, 19] to have 16 channels, but got 24 channels instead
```